In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime
from dash import Dash, html, dcc, Output, Input
pd.options.display.float_format = '{:,.0f}'.format
import glob

In [2]:
lista = lista = glob.glob("D:/Zaeli/Analise Cliente Produto Comercial/Consolidado/Recientes/*.csv")
ventas = []

In [3]:
for archivo in lista:
    data = pd.read_csv(archivo, header=0, sep = ';', parse_dates = ['Ano/Mes'], dtype = {'CodGer':str,'CODSAP':str ,'CodRepresentante':str, 'Cli&Rep':str})
    data = data.rename(columns={'Ano/Mes':'Fecha', 'CODSAP' : 'Material'})
    ventas.append(data)
ventas = pd.concat(ventas, ignore_index = True)

Actualización por inflación

In [4]:
inflacion = pd.read_excel("D:/Zaeli/BI/Archivos Compartidos/Inflacion.xlsx", header=3, sheet_name = 'Hoja1')
inflacion = inflacion.rename(columns= {'DataKey':'ChaveData'})
ventasi = pd.merge(ventas, inflacion)
ventasi['ReceitaBrutaIn'] = ventasi.ReceitaBruta * (1 + ventasi.InfacaoAcum)
ventasi['Verbas'] = ventasi.Bonif + ventasi.DescConcedidos + ventasi.Impostos + ventasi.Frete + ventasi.Comis - ventasi.Devol - ventasi.Troca
ventasi['ReceitaLiquida'] = ventasi.ReceitaBruta - ventasi.Verbas
ventasi['ReceitaLiquidaIn'] = ventasi.ReceitaLiquida * (1 + ventasi.InfacaoAcum)
ventasi['MargenBruto'] = ventasi.ReceitaLiquida - ventasi.CPV
ventasi['MargenBrutoIn'] = ventasi.MargenBruto * (1 + ventasi.InfacaoAcum)
ventasi = ventasi.drop(['PesoBruto', 'ReceitaBruta', 'Bonif', 'DescConcedidos', 'Impostos', 'Comis', 'CPV', 'Devol', 'Troca',                                                                       'Inflacaomes','InfacaoAcum', 'Verbas', 'MargenBruto', 'ReceitaLiquida'], axis=1)
ventasi = ventasi.rename(columns={
    'ReceitaBrutaIn' : 'ReceitaBruta',
    'ReceitaLiquidaIn' : 'ReceitaLiquida',
    'MargenBrutoIn' : 'Margen'
})

Carga del cadastro de productos

In [5]:
lineas = pd.read_csv("D:/Zaeli/PrevisionDemanda/lineasProducto.csv", header=0, sep=';')
lineas = lineas.rename(columns={
    'COD SAP':'Material',
    'Linha':'Linea',
    'Produto':'Producto'
})
lineas['Material'] = lineas['Material'].astype(str)
ventasP = pd.merge(ventasi, lineas)
ventasP = ventasP.drop(['Cli&Rep', 'Tipo'], axis=1)

Carga del cadastro de clientes

In [6]:
ListadoClientes = pd.read_excel("D:/Zaeli/BI/Archivos Compartidos/Clientes.xlsx",sheet_name='Listado')
ListadoClientes = ListadoClientes.drop(['Cidade', 'CodEstado', 'Estado', 'CidadeEstado'], axis=1)
ventasC = pd.merge(ventasP, ListadoClientes)

Consolidacion por mes, linea y municipio

In [7]:
ventasM = ventasC.groupby([pd.Grouper(freq='M', key = 'Fecha'), 'Fecha' ,'CodigoMunicipio', 'Linea'], as_index=False).agg({
    'ReceitaBruta':'sum',
    'ReceitaLiquida' : 'sum',
    'PesoLiq' :'sum',
    'Volume' : 'sum',
    'Margen': 'sum'
})

In [8]:
ventasM['MesAno'] = ventasM['Fecha'].dt.strftime("%d/%y")
ventasM['Ano'] = ventasM['Fecha'].dt.strftime("%y")

In [9]:
ventasM = ventasM.query('Volume >1 & ReceitaBruta >0')

In [10]:
ventasM= ventasM.rename(columns={'PesoLiq' : 'Peso'})

In [11]:
ventasL = ventasM.groupby([pd.Grouper(freq='M', key = 'Fecha'), 'Fecha', 'MesAno', 'Ano',  'CodigoMunicipio'], as_index=False).agg({
    'ReceitaBruta':'sum',
    'ReceitaLiquida' : 'sum',
    'Peso' :'sum',
    'Volume' : 'sum',
    'Margen': 'sum'
})

Preparacion y carga del archivo de municipios

In [12]:
datosMun = pd.read_csv("D:/Zaeli/PrevisionDemanda/DatosMunicipios.csv", header=0, sep = ';')
datosMun = datosMun.rename(columns = {
    'CódigoMunicípio':'CodigoMunicipio',
    'Área Territorial 0 km² [2020]':'Area_km2',
    'População estimada 0 pessoas [2021]': 'Poblacion',
    'Densidade demográfica 0 hab/km² [2010]': 'Habitantes_km2',
    'Escolarização6a14%': 'PorcEscolarizacion6a14',
    'IDHM ' : 'IDHM',
    'Mortalidade infantil' : 'Mortalidad_Infantil'
})
datosMun.loc[datosMun['IDHM'] < 0.49 , 'Desarrollo'] = 'Muy Bajo'
datosMun.loc[(datosMun['IDHM'] >= 0.49) & (datosMun['IDHM'] < 0.599) , 'Desarrollo'] = 'Bajo'
datosMun.loc[(datosMun['IDHM'] >= 0.599) & (datosMun['IDHM'] < 0.699) , 'Desarrollo'] = 'Medio'
datosMun.loc[(datosMun['IDHM'] >= 0.699) & (datosMun['IDHM'] <= 0.799) , 'Desarrollo'] = 'Alto'
datosMun.loc[datosMun['IDHM'] >= 0.80 , 'Desarrollo'] = 'Muy Alto'
datosMun.loc[datosMun['IDHM'] < 0.49 , 'DesarrolloNum'] = '1'
datosMun.loc[(datosMun['IDHM'] >= 0.49) & (datosMun['IDHM'] < 0.599) , 'DesarrolloNum'] = '2'
datosMun.loc[(datosMun['IDHM'] >= 0.599) & (datosMun['IDHM'] < 0.699) , 'DesarrolloNum'] = '3'
datosMun.loc[(datosMun['IDHM'] >= 0.699) & (datosMun['IDHM'] <= 0.799) , 'DesarrolloNum'] = '4'
datosMun.loc[datosMun['IDHM'] >= 0.80 , 'DesarrolloNum'] = '5'

In [13]:
col_minimercados = np.argwhere(datosMun.columns.isin(['MiniMercDemais','MiniMercEPP', 'MiniMercME', 'MiniMercMEI'])).ravel()
col_supermercados = np.argwhere(datosMun.columns.isin(['SupermercadoDemais','SupermercadoEPP', 'SupermercadoME', 'SupermercadoMEI'])).ravel()
col_conveniencias = np.argwhere(datosMun.columns.isin(['ConvenienciaDemais', 'ConvenienciaEPP', 'ConvenienciaME','ConvenienciaMEI'])).ravel()
col_hipermercados =  np.argwhere(datosMun.columns.isin(['HiperDemais', 'HiperEPP'])).ravel()
col_atacados = np.argwhere(datosMun.columns.isin(['AtacadoDemais', 'AtacadoEPP'])).ravel()

In [14]:
datosMun['Minimercados'] = datosMun.iloc[:,col_minimercados].sum(axis=1)
datosMun['Supermercados'] = datosMun.iloc[:,col_supermercados].sum(axis=1)
datosMun['Conveniencias'] = datosMun.iloc[:,col_conveniencias].sum(axis=1)
datosMun['Hipermercados'] = datosMun.iloc[:,col_hipermercados].sum(axis=1)
datosMun['Atacados'] = datosMun.iloc[:,col_atacados].sum(axis=1)

In [15]:
datosMun.loc[(datosMun.Hipermercados + datosMun.Atacados) == 0, 'TipoComercios'] = 'Sin Hipermercados ni Atacados'
datosMun.loc[((datosMun.Hipermercados + datosMun.Atacados) >0) & ((datosMun.Hipermercados + datosMun.Atacados) <= 5), 'TipoComercios' ] = '< 5 Atacados e Hipermercados'
datosMun.loc[((datosMun.Hipermercados + datosMun.Atacados) >5) & ((datosMun.Hipermercados + datosMun.Atacados) <= 10), 'TipoComercios' ] = '< 10 Atacados e Hipermercados'
datosMun.loc[(datosMun.Hipermercados + datosMun.Atacados) >10 , 'TipoComercios' ] = '> 10 Atacados e Hipermercados'
datosMun.loc[(datosMun.Hipermercados + datosMun.Atacados) == 0, 'TipoComerciosNum'] = '1'
datosMun.loc[((datosMun.Hipermercados + datosMun.Atacados) >0) & ((datosMun.Hipermercados + datosMun.Atacados) <= 5), 'TipoComerciosNum' ] = '2'
datosMun.loc[((datosMun.Hipermercados + datosMun.Atacados) >5) & ((datosMun.Hipermercados + datosMun.Atacados) <= 10), 'TipoComerciosNum' ] = '3'
datosMun.loc[(datosMun.Hipermercados + datosMun.Atacados) >10 , 'TipoComerciosNum' ] = '4'

In [16]:
datosMun = datosMun.drop(['VAAgro', 'VAIndustria','VAServicos', 'VAAdmPublica','VATotal', 'Impostos', 'PBI','Atividade secundaria', 'Atividade terciaria', 'Area_km2',
                          'Habitantes_km2', 'PorcEscolarizacion6a14', 'Mortalidad_Infantil', 'ReceitaPrefeitura', 'DespesasPrefeitura','Mortalidad_Infantil', 'ReceitaPrefeitura', 'DespesasPrefeitura','HiperDemais', 'HiperEPP', 'HiperME', 'HiperMEI', 'MiniMercDemais', 'MiniMercEPP', 'MiniMercME', 'MiniMercMEI', 'SupermercadoDemais', 'SupermercadoEPP', 'SupermercadoME', 'SupermercadoMEI', 'ConvenienciaDemais', 'ConvenienciaEPP', 'ConvenienciaME', 'ConvenienciaMEI', 'AtacadoDemais', 'AtacadoEPP', 'AtacadoME', 'AtacadoMEI'
                          ], axis=1)

Variables de seleccion

In [17]:
ventas_tot_mun = pd.merge(ventasL, datosMun)

In [18]:
ventas_tot_mun = ventas_tot_mun.drop(columns=['CodigoMunicipio'], axis=1)

In [94]:
estado = 'Ceara'
mes = '03/22'
municipio = 'Apucarana'

In [95]:
ventas_tot_mun_22 = ventas_tot_mun.query('Estado == @estado & MesAno == @mes & Municipio != @municipio')

In [96]:
ventas_tot_mun_22 = ventas_tot_mun_22.sort_values('DesarrolloNum')

In [97]:
opciones_estado = list(ventas_tot_mun_22.Estado.unique())
opciones_municipio = list(ventas_tot_mun_22.Municipio.unique())
opciones_tipoComercio = list(ventas_tot_mun_22.TipoComercios.unique())
opciones_desarrollo = list(ventas_tot_mun_22.Desarrollo.unique())

Graficos del total de ventas por municipio

In [98]:
fig = px.scatter(ventas_tot_mun_22,
                 x = 'Peso',
                 y = 'Margen',
                 category_orders = {'Desarrollo': opciones_desarrollo},
                 size = 'Poblacion',
                 hover_name= 'Municipio',
                 hover_data= {'Estado' : True,
                              'AtividadePrincipal' : True,
                              'Margen' : False,
                              'Peso' : ':,.0f',
                              'PBIperCapita' : ':,.0f',
                              'Poblacion' : ':,.0f',
                              'ReceitaBruta' : ':,.0f'},
                 color = 'TipoComercios',
                 title= f'Peso y margen mes {mes}'
                 )
fig.show()

In [99]:
fig2 = px.scatter(ventas_tot_mun_22,
                 x = 'Peso',
                 y = 'Margen',
                  range_x=[0, ventas_tot_mun_22.Peso.max()],
                  range_y=[0, ventas_tot_mun_22.Margen.max()],
                 category_orders = {'Desarrollo': opciones_desarrollo},
                 size = 'Poblacion',
                  size_max= 60,
                 hover_name= 'Municipio',
                 hover_data= {'Estado' : False,
                              'AtividadePrincipal' : True,
                              'Margen' : False,
                              'Peso' : ':,.0f',
                              'PBIperCapita' : ':,.0f',
                              'Poblacion' : ':,.0f',
                              'IDHM' : ':,.2f',
                              'ReceitaBruta' : ':,.0f'},
                 color = 'Desarrollo',
                 title= f'Peso y margen mes {mes} no estado de {estado}'
                 )
fig2.show()